In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
import math

In [2]:
d_labitems = pd.read_csv('data/mimic-iv-0.4/hosp/d_labitems.csv.gz', compression='gzip')

In [5]:
admissions_sample = pd.read_pickle("admissions_sample.pkl")

## Pull Chart and Lab data only where it matches our sample patient list

In [6]:
# # We have Decided not to use Chart data after exploring it further
# # However, it still comes up in our discussions, so keeping this for now
# # Will need to update this code based on other updates we've made

# %%time #run this for next time

# chunksize = 3*(10 ** 6)
# counter=0
# chartevents_sample = []
# for chunk in pd.read_csv('data/mimic-iv-0.4/icu/chartevents.csv.gz', compression='gzip', chunksize=chunksize):
#     chartevents_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
#     counter+=1
#     print(f'chunk {counter} processed')

# chartevents_final = pd.concat(chartevents_sample)



In [7]:
# chartevents_final.to_pickle("chartevents_final.pkl")

In [8]:
# chartevents_final = pd.read_pickle("chartevents_final.pkl")

In [9]:
# chartevents_final.shape

Load Lab Events table

In [10]:
%%time

chunksize = 3*(10 ** 6)
counter=0
lab_events_sample = []
for chunk in pd.read_csv('data/mimic-iv-0.4/hosp/labevents.csv.gz', compression='gzip', chunksize=chunksize):
    lab_events_sample.append(chunk[chunk['subject_id'].isin(list(admissions_sample.subject_id))])
    counter+=1
    print(f'chunk {counter} processed')

lab_events_final = pd.concat(lab_events_sample)
# 5k patients
# 3min 52s

# 10k patients
# 4min 6s

# 15k patients
# 4min 27s



chunk 1 processed
chunk 2 processed
chunk 3 processed


<string>:2: DtypeWarning: Columns (7,9,12) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 4 processed
chunk 5 processed
chunk 6 processed
chunk 7 processed
chunk 8 processed
chunk 9 processed


<string>:2: DtypeWarning: Columns (9,12) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 10 processed


...

chunk 11 processed
chunk 12 processed
chunk 13 processed
chunk 14 processed
chunk 15 processed


<string>:2: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 16 processed
chunk 17 processed
chunk 18 processed
chunk 19 processed
chunk 20 processed
chunk 21 processed
chunk 22 processed
chunk 23 processed


<string>:2: DtypeWarning: Columns (7,9) have mixed types.Specify dtype option on import or set low_memory=False.


chunk 24 processed
chunk 25 processed
chunk 26 processed
chunk 27 processed
chunk 28 processed
chunk 29 processed
chunk 30 processed
chunk 31 processed
chunk 32 processed
chunk 33 processed
chunk 34 processed
chunk 35 processed
chunk 36 processed
chunk 37 processed
chunk 38 processed
chunk 39 processed
chunk 40 processed
chunk 41 processed
CPU times: user 6min 45s, sys: 48.3 s, total: 7min 34s
Wall time: 9min 35s


In [11]:
lab_events_final.to_pickle("lab_events_final.pkl")

In [12]:
# lab_events_final = pd.read_pickle("lab_events_final.pkl")

In [13]:
lab_events_final.shape

(6765210, 15)

## Manipulating the Data

Take only records related to our sample patients

Filter lab events based on first hospital visit of our sample population

In [ ]:
# Can we do this filtering when generating lab data?
# For now let's not as we're discussing using different hospitalizations from the same subjects

%time lab_events_sampled = lab_events_final[lab_events_final['hadm_id'].isin(admissions_sample.hadm_id)]


Check how many patients have had each test done

In [ ]:
lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)

Most common tests performed on patients with pos diagnosis

In [ ]:
lab_events_sampled[lab_events_sampled['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(10)


Item Black List

In [ ]:
item_black_list = [50920]
# 50920 - test results are text in comments - need to come back and figure out how to handle this


Take only the X most commonly performed tests

In [ ]:
top_test_num = 40

lab_events_valid = lab_events_sampled[~lab_events_sampled['itemid'].isin(item_black_list)]

# On patients with pos diagnosis
itemid_sub_sample = lab_events_valid[lab_events_valid['subject_id'].isin(list(admissions_sample[admissions_sample['pos'] == 1].subject_id))].groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()
# On all sample patients
# itemid_sub_sample = lab_events_sampled.groupby('itemid')['subject_id'].nunique().sort_values(ascending = False).head(top_test_num).reset_index().rename(columns = {'index' : 'itemid'}).itemid.to_list()

lab_events_sampled_sub = lab_events_sampled[lab_events_sampled['itemid'].isin(itemid_sub_sample)]


## Data Cleaning

Make a separate table where we run some of the forest models on the Text data?

Special updates for Urine sample tests

In [ ]:
# Other invalid value imputation
# For now set them all to normal/most common value
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin([np.nan, 'ERROR', 'UNABLE TO REPORT'])), ['value', 'valuenum']] = 1.1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>21.8'])), ['value', 'valuenum']] = 22
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>19.2'])), ['value', 'valuenum']] = 20

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['>150', '>150.0'])), ['value', 'valuenum']] = 175

# Should we just delete these?
# For 5k patients there are 6 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51274) &  (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = 11

# Should we just delete these?
# For 5k patients there are 1 records here
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['UNABLE TO REPORT'])), ['value', 'valuenum']] = 31

# Clumsy..
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) &  (lab_events_sampled_sub['value'].isin(['34..3'])), ['value', 'valuenum']] = 34.3


More Data Cleaning

In [ ]:
%%time

# Can probably make a helper function combining some of the below mapping

# 51466
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE', 'LRG'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51514
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', 'NORMAL.  ', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>8'])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51514) & (lab_events_sampled_sub['value'].isin(['>12.  ', '>12*.  '])), ['value', 'valuenum']] = 15

# 51464
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 3

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51464) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51487
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['value'].isin(['POS.  ', 'POS', 'POS*.  '])), ['value', 'valuenum']] = 1

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51487) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51492
# Should Pos be marked abnormal?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['___'])), ['value', 'valuenum']] = np.nan
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' '])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].str.contains('TR.')), ['value', 'valuenum']] = 10
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>300'])), ['value', 'valuenum']] = 350
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['value'].isin(['>600.  ', '>600*.  '])), ['value', 'valuenum']] = 700

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51492) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51486
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['NEG.  ', 'NEG', ' ', 'N'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['TR', 'TR.  ', 'TR*.  '])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['SM', 'SM .  ', 'SM*.  '])), ['value', 'valuenum']] = 2
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['MOD', 'MOD.  ', 'MOD*.  '])), ['value', 'valuenum']] = 3
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['value'].isin(['LG', 'LG.  ', 'LG*.  ', 'LGE'])), ['value', 'valuenum']] = 4

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51486) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# 51506
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin([np.nan])), ['value']] = lab_events_sampled_sub['comments']
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Clear.  ', 'CLEAR.  ', 'Clear', ' ', 'CLEAR'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['SlHazy'])), ['value', 'valuenum']] = 0.5
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Hazy.  ', 'Hazy*.  ', 'Hazy', 'HAZY', 'HAZY*.  ', 'Slcldy', 'SLCLOUDY'])), ['value', 'valuenum']] = 1
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['value'].isin(['Cloudy*.  ', 'Cloudy.  ', 'Cloudy', 'CLO', 'CLOUDY', 'CLOU', 'CLOUDY*.  '])), ['value', 'valuenum']] = 2

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51506) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0


Additional Data Cleaning

In [ ]:
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin([np.nan, ' ', 'TR', 'NEG'])), ['value', 'valuenum']] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_lower'].isnull()), 'ref_range_lower'] = 0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['ref_range_upper'].isnull()), 'ref_range_upper'] = 0

# What are good replacement values for these?
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].isin(['>80'])), ['value', 'valuenum']] = 120
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].isin(['>1000'])), ['value', 'valuenum']] = 1250

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['<=1.005'])), ['value', 'valuenum']] = 1.000
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.030', '.1.030'])), ['value', 'valuenum']] = 1.035
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.050'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['comments'].isin(['>1.050*.'])), ['value', 'valuenum']] = 1.055
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>=1.035'])), ['value', 'valuenum']] = 1.040
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 1.015

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51237) & (lab_events_sampled_sub['value'].isin(['>13.4'])), ['value', 'valuenum']] = 15

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51491) & (lab_events_sampled_sub['value'].isin([' '])), ['value', 'valuenum']] = 6.0

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51498) & (lab_events_sampled_sub['value'].isin(['>1.070'])), ['value', 'valuenum']] = 1.080

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51275) & (lab_events_sampled_sub['value'].isin(['ERROR'])), ['value', 'valuenum']] = np.nan

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51484) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'
lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51478) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'

lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == 51466) & (lab_events_sampled_sub['value'].astype(float) > 0), 'flag'] = 'abnormal'


In [ ]:
# Check on differences between value and valuenum columns

lab_events_sampled_sub.loc[((lab_events_sampled_sub['value'].astype(str) + str('.0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str) + str('0')) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str)) 
                           & ((lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str) + str('0')) 
                           & ((str('0') + lab_events_sampled_sub['value'].astype(str)) != lab_events_sampled_sub['valuenum'].astype(str))
                           & ((lab_events_sampled_sub['value'].astype(str)) != round(lab_events_sampled_sub['valuenum'], 10).astype(str)), ]


## Code to look into specific test values for cleaning

In [ ]:
# lab_events_sampled_sub[lab_events_sampled_sub['value'] == 'N']

In [ ]:
# # Used to investigate bad data
# pd.set_option('display.max_rows', 200)

# bad_id = 51486

# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id].value.value_counts().reset_index().sort_values(by = 'index')
# # lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['value', 'comments']].drop_duplicates()
# lab_events_sampled_sub[lab_events_sampled_sub['itemid'] == bad_id][['flag', 'comments', 'value', 'valuenum', 'ref_range_lower', 'ref_range_upper']].drop_duplicates()
# # lab_events_sampled_sub.loc[(lab_events_sampled_sub['itemid'] == bad_id) & (lab_events_sampled_sub['value'].isnull()), ][['value', 'valuenum', 'comments']].drop_duplicates()


In [ ]:
# pd.set_option('display.max_rows', 20)

## Get Aggregate Test Values

In [ ]:
%time lab_events_sampled_sub_grouped = lab_events_sampled_sub.groupby(['subject_id', 'hadm_id', 'itemid']).agg({'itemid' : ['count'], 'valuenum' : ['min', 'max', 'mean'], 'flag' : ['count'], 'ref_range_lower' : ['min'], 'ref_range_upper' : ['min']})
# 'comments' : ['unique'], 


In [ ]:
lab_events_sampled_sub_grouped.reset_index(inplace = True)
lab_events_sampled_sub_grouped.columns = ['subject_id', 'hadm_id', 'itemid', 'count', 'min', 'max', 'mean', 'abn_percent', 'range_min', 'range_max']
# 'comments', 


In [ ]:
missing_vals = lab_events_sampled_sub_grouped.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

In [ ]:
lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['min'].isnull(), ].itemid.value_counts()

# lab_events_sampled_sub_grouped.loc[(lab_events_sampled_sub_grouped['min'].isnull()) & (lab_events_sampled_sub_grouped['itemid'] == 51003), ]#.subject_id.nunique()
# # lab_events_sampled_sub_grouped.subject_id.nunique()

# lab_events_sampled_sub_grouped.loc[lab_events_sampled_sub_grouped['subject_id'] == 10002264, ]


In [ ]:
lab_events_sampled_sub_grouped['below_min'] = np.where(lab_events_sampled_sub_grouped['min'] < lab_events_sampled_sub_grouped['range_min'], lab_events_sampled_sub_grouped['range_min'] - lab_events_sampled_sub_grouped['min'], 0)
lab_events_sampled_sub_grouped['above_max'] = np.where(lab_events_sampled_sub_grouped['max'] > lab_events_sampled_sub_grouped['range_max'], lab_events_sampled_sub_grouped['max'] - lab_events_sampled_sub_grouped['range_max'], 0)
lab_events_sampled_sub_grouped['abn_percent'] = lab_events_sampled_sub_grouped['abn_percent'] / lab_events_sampled_sub_grouped['count']


## Merge Admissions data so we can group by ethnicity/gender to grab average range min and max

In [ ]:
# Here is where we merge Admissions

# lab_events_sampled_sub_grouped.merge(d_icd_diagnoses, left_index = True, left_on = "index", right_on = 'icd_code', how = 'left')


In [ ]:
# For now aggregate all patients together

lab_range_min_max = lab_events_sampled_sub_grouped.groupby('itemid').agg({'range_min' : 'mean', 'range_max' : 'mean'})
lab_range_min_max.reset_index(inplace = True)

lab_range_dic = dict(zip(lab_range_min_max['itemid'], zip(lab_range_min_max['range_min'], lab_range_min_max['range_max'])))
lab_range_dic


Pivot the table so we have feature columns related to test results

In [ ]:
lab_events_sampled_sub_grouped['new_index'] = list(zip(lab_events_sampled_sub_grouped['subject_id'], lab_events_sampled_sub_grouped['hadm_id']))


In [ ]:
%time lab_events_sampled_pivot = lab_events_sampled_sub_grouped.pivot(index = 'new_index', columns = 'itemid', values = ['min', 'max', 'mean', 'abn_percent', 'below_min', 'above_max'])
# 'comments', 


In [ ]:
lab_events_sampled_pivot = lab_events_sampled_pivot.reset_index()
lab_events_sampled_pivot['subject_id'], lab_events_sampled_pivot['hadm_id'] = zip(*lab_events_sampled_pivot['new_index'])
lab_events_sampled_pivot.drop(['new_index'], axis = 1, inplace = True)


In [ ]:
missing_vals = lab_events_sampled_pivot.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

## Start Data Imputation

Copy table, retain original for Random Forest

In [ ]:
lab_events_impute = lab_events_sampled_pivot.copy()

In [ ]:
d_labitems[d_labitems['itemid'].isin(itemid_sub_sample)]

In [ ]:
%%time

np.random.seed(0)

for labitem in lab_range_dic:
    for ind in lab_events_impute[lab_events_impute[('mean', labitem)].isnull()].index:
        val_max = lab_range_dic[labitem][1]
        val_min = lab_range_dic[labitem][0]

        val_ave = (val_max + val_min) / 2
        val_std = (val_max - val_ave) * .333

        ran_vals = np.random.normal(val_ave, val_std, 50)
        impute_min = min(ran_vals)
        impute_max = max(ran_vals)
        impute_mean = np.mean(ran_vals)
        
        val_min = lab_events_impute[('min', labitem)][ind]
        val_max = lab_events_impute[('max', labitem)][ind]
        val_mean = lab_events_impute[('mean', labitem)][ind]
        
        lab_events_impute[('min', labitem)][ind] = np.where(np.isnan(val_min), impute_min, val_min)
        lab_events_impute[('max', labitem)][ind] = np.where(np.isnan(val_max), impute_max, val_max)
        lab_events_impute[('mean', labitem)][ind] = np.where(np.isnan(val_mean), impute_mean, val_mean)
#         patient[('mean', labitem)].replace(np.nan, impute_mean)
#         patient[('abn_count', labitem)].replace(np.nan, 0)
# 5k patients
# 10 items - 2.5s
# 20 items - 6s
# 30 items - 28.6s

# 10k patients
# 40 items - 1m 34s

In [ ]:
for lab_itemid in lab_range_dic:
    lab_events_impute[( 'above_max', lab_itemid)] = lab_events_impute[( 'above_max', lab_itemid)].replace(np.nan, 0)
    lab_events_impute[( 'below_min', lab_itemid)] = lab_events_impute[( 'below_min', lab_itemid)].replace(np.nan, 0)
    lab_events_impute[( 'abn_percent', lab_itemid)] = lab_events_impute[( 'abn_percent', lab_itemid)].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))


In [ ]:
missing_vals = lab_events_impute.isnull().sum(axis = 0) 
missing_vals[missing_vals > 0].sort_values()

In [ ]:
lab_events_impute

In [ ]:
lab_events_impute_KNN = lab_events_sampled_pivot.copy()

In [ ]:
%%time

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=math.trunc((lab_events_impute_KNN.shape[0] ** 0.5)))
lab_events_impute_KNN_imputed = imputer.fit_transform(lab_events_impute_KNN)

# 3min 8s

In [ ]:
lab_events_impute_KNN_imputed = pd.DataFrame(lab_events_impute_KNN_imputed)
lab_events_impute_KNN_imputed.columns = lab_events_impute_KNN.columns
lab_events_impute_KNN_imputed

In [ ]:
for lab_itemid in lab_range_dic:
    lab_events_impute_KNN_imputed[( 'above_max', lab_itemid)] = lab_events_impute_KNN_imputed[( 'above_max', lab_itemid)].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'below_min', lab_itemid)] = lab_events_impute_KNN_imputed[( 'below_min', lab_itemid)].replace(np.nan, 0)
    lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid)] = lab_events_impute_KNN_imputed[( 'abn_percent', lab_itemid)].replace(np.nan, 0)
#     lab_events_sampled_pivot[( 'comments', lab_itemid)] = lab_events_sampled_pivot[( 'comments', lab_itemid)].replace(np.nan, np.array(np.nan))



## Standardize the Data for use in Models

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [ ]:
labs_scaled = lab_events_impute
labs_scaled_KNN = lab_events_impute_KNN_imputed
labs_scaled_missing_data = lab_events_sampled_pivot

In [ ]:
scale_cols = ~labs_scaled.columns.isin([('subject_id', ''), ('hadm_id', '')])

scaler.fit(labs_scaled.loc[:, scale_cols])
labs_scaled.loc[:, scale_cols] = scaler.transform(labs_scaled.loc[:, scale_cols])

scaler.fit(labs_scaled_KNN.loc[:, scale_cols])
labs_scaled_KNN.loc[:, scale_cols] = scaler.transform(labs_scaled_KNN.loc[:, scale_cols])

scaler.fit(labs_scaled_missing_data.loc[:, scale_cols])
labs_scaled_missing_data.loc[:, scale_cols] = scaler.transform(labs_scaled_missing_data.loc[:, scale_cols])


## Save Lab Results for Models

In [ ]:
# Save non-imputed file as well for forest models

pos_final_test = labs_scaled.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')
pos_KNN_final_test = labs_scaled_KNN.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')
pos_final_missing_data_test = labs_scaled_missing_data.merge(admissions_sample, how='left', left_on='hadm_id', right_on='hadm_id')


In [ ]:
pos_final_test.to_csv('pos_final_test.csv.gz', compression="gzip")
pos_KNN_final_test.to_csv('pos_KNN_final_test.csv.gz', compression="gzip")
pos_final_missing_data_test.to_csv('pos_final_missing_data_test.csv.gz', compression="gzip")


## Check for Multicollinearity

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

In [ ]:
%%time

labs_scaled_features = labs_scaled.loc[:, ('min', 50868) : ('above_max', 51484)]
add_constant(labs_scaled_features)

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features.values, i) 
               for i in range(labs_scaled_features.shape[1])], 
              index=labs_scaled_features.columns).sort_values(ascending = False)

high_vif.head(20)


In [ ]:
labs_scaled_features_limited = labs_scaled_features.copy()
labs_scaled_features_limited.drop([('above_max', 51484),
                                  ('above_max', 51478),
                                  ('above_max', 51466),
                                  ('above_max', 51464),
                                  ('max', 51003),
                                  ('max', 50878),
                                  ('max', 50861),
                                  ('max', 51237),
                                  ('mean', 51248),
                                  ('max', 50931),
                                  ('mean', 51222),
                                  ('max', 51274),
                                  ('mean', 51279),
                                  ('mean', 51274),
                                  ('mean', 51478),
                                  ('mean', 50885),
                                  ('mean', 51250),
                                  ('max', 50911),
                                  ('mean', 51003),
                                  ('mean', 51466),
                                  ('mean', 51484),
                                  ('mean', 51486),
                                  ('mean', 51277),
                                  ('mean', 51221),
                                  ('max', 50912),
                                  ('max', 51275),
                                  ('mean', 51492),
                                  ('mean', 51514),
                                  ('mean', 51006),
                                  ('min', 51222),
                                  ('max', 50960)], axis = 1, inplace = True)


In [ ]:
%%time

high_vif = pd.Series([variance_inflation_factor(labs_scaled_features_limited.values, i) 
               for i in range(labs_scaled_features_limited.shape[1])], 
              index=labs_scaled_features_limited.columns).sort_values(ascending = False)

high_vif.head(20)
